# First Checkpoint
### Predicting Pneumonia from X-Ray image

Jimena Salinas Valdespino, Santiago Segovia Baquero, Stephania Tello Zamudio, Ivanna Rodríguez Lobo

In [16]:
import os
import pandas as pd

from torch.utils import data
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision.io import read_image

## Creating a Pytorch Dataset


In [5]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, img_dir_path, resize=False, transform=None):
        """
        Inputs:
            - csv_file (str): file path to the csv file
            - img_dir_path: directory path to the images
            - transform: Compose (a PyTorch Class) that strings together several
              transform functions (e.g. data augmentation steps)
        """
        self.img_labels = pd.read_csv(csv_file, skiprows=1, header=None)
        self.img_dir = img_dir_path
        self.transform = transform
        self.resize = resize
        self.dimensions = self.get_dimensions()

    def __len__(self):
        """
        Returns: (int) length of your dataset
        """
        return len(self.img_labels)

    def __getitem__(self, idx):
        """
        Loads and returns your sample (the image and the label) at the
        specified index

        Parameter: idx (int): index of interest

        Returns: image, label
        """
        img_path =  os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        
        # read the image
        image = read_image(img_path)

        # get the label
        label = self.img_labels.iloc[idx, 1]

        # apply transformations to image
        if self.transform:
            image = self.transform(image)

        return image, label
    
    def get_dimensions(self):
        """
        This method creates a dictionary with the unique combinations of heightxwidth
        for each image in the dataset.

        returns a dictionary with dimensions as keys and the number of images
            with that dimension as values
        """
        dimensions = {}
        for index in range(len(self.img_labels)):
            image = self[index][0]
            if self.resize:
                image = self.resize_image(image)
            _, height, width = image.shape
            dimensions[(height,width)] = dimensions.get((height,width),0) + 1

        return dimensions
    
    def resize_image(self,image):
        """
        If the resize parameter==True, then all the images are
        converted to a 150x150 size.

        returns the resized image
        """
        transform = T.Resize((150,150))
        
        return transform(image)

Once we implemented the Dataset class, we create one object per Dataset:

In [6]:
train_data = CustomImageDataset(csv_file = '../data/data_train.csv',
                                img_dir_path = '../data')

val_data = CustomImageDataset(csv_file = '../data/data_val.csv',
                              img_dir_path = '../data')

test_data = CustomImageDataset(csv_file = '../data/data_test.csv',
                               img_dir_path = '../data')

The labels for each one of the datasets are the following:

In [7]:
# Train dataset
train_data.img_labels.iloc[:,1].value_counts()

1    3875
0    1341
Name: 1, dtype: int64

In [8]:
# Validation dataset
val_data.img_labels.iloc[:,1].value_counts()

0    8
1    8
Name: 1, dtype: int64

In [9]:
# Test dataset
test_data.img_labels.iloc[:,1].value_counts()

1    390
0    234
Name: 1, dtype: int64

Inside our CustomImageDataset class we defined a method to compute a count of each height x width combination in our data set. The cells below display our results:

In [10]:
train_data.dimensions
len(train_data.dimensions)

4366

There are 4,366 unique height x width combinations in our training data set.

In [11]:
test_data.dimensions
len(test_data.dimensions)

598

There are 598 unique height x width combinations in our testing data set.

In [12]:
val_data.dimensions
len(val_data.dimensions)

16

There are 16 unique height x width combinations in our validation data set.

Given that there are different sizes, and some very large images, we want to standarize the size of all images. We do this by passing a `resize` boolean parameter to our `CustomImageDataset` class:

In [13]:
train_data = CustomImageDataset(csv_file = '../data/data_train.csv',
                                img_dir_path = '../data',
                                resize=True)

val_data = CustomImageDataset(csv_file = '../data/data_val.csv',
                              img_dir_path = '../data',
                              resize=True)

test_data = CustomImageDataset(csv_file = '../data/data_test.csv',
                               img_dir_path = '../data',
                               resize=True)

/home/stello/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/stello/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be cons

As a check, we can call on the `dictionary` attribute for each one of our datasets:

In [14]:
train_data.dimensions

{(150, 150): 5216}

In [12]:
val_data.dimensions

{(150, 150): 16}

In [13]:
test_data.dimensions

{(150, 150): 624}

## Image Augmentation

In order to avoid overfitting, we need to do image augmentation for our training
dataset. We do this below. We decided to augment the images following some 
examples of people who worked with this dataset in Kaggle.

The transformations we used are the following: rotate the image by 30 degrees, zoom into the image by 20%, flip the image horizontally, increase the image's sharpness, and change the color depth of the image.

In [17]:
train_transform = T.Compose([
    T.RandomRotation(30),
    T.RandomResizedCrop(size=(150, 150), scale=(0.8, 1.2)),
    T.RandomHorizontalFlip(),
    T.RandomAdjustSharpness(sharpness_factor=2),
    T.RandomPosterize(bits=4),
    T.Grayscale(1)
])

We then apply the above transformations to our training dataset below.

In [18]:
# Apply the transforms to the training dataset
train_data = CustomImageDataset(csv_file = '../data/data_train.csv',
                                img_dir_path = '../data',
                                resize=True,
                                transform=train_transform)

/home/stello/.local/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


## Creating our DataLoader

Below, we create our DataLoader. The purpose of doing this is to load our data in
batches to fit and test our model.

In [16]:
train_dataloader = DataLoader(train_data, 
                              batch_size=64, 
                              shuffle=True)

val_dataloader = DataLoader(val_data, 
                            batch_size=64, 
                            shuffle=True)

test_dataloader = DataLoader(test_data, 
                             batch_size=64, 
                             shuffle=True)